# Setup Git-crypt with empty directory

In [36]:
DEBUG=True

# feature flags to debug
SETUP=False
UNLOCK=True
INCR_EXPERIMENTAL=False
#ref: https://stephanosterburg.gitbook.io/scrapbook/data-science/jupiter-notebook-tips-and-tricks

In [37]:
%alias gcrypt docker run -t -v %s:/repo xueshanf/git-crypt git-crypt %s
%store gcrypt
if DEBUG:
    %alias

Alias stored: gcrypt (docker run -t -v %s:/repo xueshanf/git-crypt git-crypt %s)
Total number of aliases: 13


In [38]:
if SETUP:
    !python3 -m pip install ipywidgets
    !python3 -m pip install papermill
    %gcrypt /Users/kenneyhe/Desktop/data-science status -e
    %gcrypt /Users/kenneyhe/Desktop/data-science init
    %gcrypt /Users/kenneyhe/Desktop/data-science export-key git-crypt-ds-key
    !mv git-crypt-ds-key ..
    !git add .gitattributes
    # if using osx to add
    !brew install --build-from-source git-crypt
    %gcrypt /Users/kenneyhe/Desktop/data-science unlock ../git-crypt-ds-key

if UNLOCK:
    !docker run -t -v $PWD:/repo  -v /Users/kenneyhe/Desktop/git-crypt-ds-key:/repo/key xueshanf/git-crypt git-crypt unlock key

In [39]:
!git commit -m "Tell git-crypt to encrypt secrets.yaml"

[ERROR] Your pre-commit configuration is unstaged.
`git add .pre-commit-config.yaml` to fix this.


In [40]:
# %load secrets.yaml
PASS="d3229f71-4047-4136-a0b8-22cbb83737a2"
USER="kenney"


In [ ]:
# %load api.py
'''
API for backup current directory to kenney/mobile docker hub repo with
dated tag
'''

import datetime
import uuid
import os
import sys
import papermill as pm

timestamp=datetime.date.today().strftime("%Y%m%d%H%sb")
UUID_STR=str(uuid.uuid1())[0:7]


# default feature flags
INCR_EXPERIMENTAL=False
DEBUG=False
RESTORE=False
DEP_NEED=False


def sync_mobile(dir_name, usern="", passw=""):
    '''
        DESCRIPTION: sync latest to the current dated tag
        PARAM: dir_list is list of directorys under Desktop
               gs://.../.. can be used instead if upload output to
               s3
    '''
    if INCR_EXPERIMENTAL:
        d_lower = dir_name.lower()
        # match backup helper convention
        tag_name ="%s-%s" % (d_lower, timestamp)

        status = os.system('''
                    docker login -u {user} --password {dpass};
                    docker tag kenney/{tname} kenney/mobile:{tname};
                    docker tag kenney/{tname} kenney/mobile:{tdir}-latest;
                    docker push kenney/mobile:{tname} && docker push kenney/mobile:{tdir}-latest;
                    '''.format(tname=tag_name, tdir=d_lower, user=usern, dpass=passw))
        if DEBUG:
            print('%s has status %d: done' % (tag_name, status))
        assert status == 0, "failed"


def backup(dir_list, is_full=False, generate=False, user="", passw=""):
    '''
        DESCRIPTION: backups the dir_lists and upload to docker
        PARAM: dir_list is list of directorys under Desktop
               gs://.../.. can be used instead if upload output to
               s3
        EXCEPTIONS: throw exception up to caller if unexpected
    '''
    for d_name in dir_list:
        uuid_str=str(uuid.uuid1())[0:7]
        output="output_%s_run_backup_%s.ipynb" % (timestamp, uuid_str)
        try:
            notebook = pm.execute_notebook("BackupHelper.ipynb",
                                output,
                                {
                                 "USER": user,
                                 "PASS": passw,
                                 "DIR": d_name,
                                 "DEBUG": False,
                                 "FULL": is_full,
                                 "GEN": generate,
                                 "progress_bar": True,
                                 "log_output": True,
                                 "report_mode": True,
                                }
                               )
            print(notebook)
        except pm.PapermillExecutionError:
            print("failed to run %s" %(d_name))
            if DEBUG:
                print(uuid_str, '=>', output)
                print(str(notebook))

            sys.exit(1)
        print('%s is done' % (d_name))

        sync_mobile(d_name)


def restore(dir_dict, user="", passw=""):
    '''
        DESCRIPTION: restores the dir_lists and upload to docker
               assuming all directories are made to be public
        PARAM: dir_dict is dictionary of directory name with tags
               under Desktop
               gs://.../.. can be used instead if upload output to
               s3
    '''
    for dir_name, tag in dir_dict.items():
        uuid_str=str(uuid.uuid1())[0:7]
        output="output_%s_run_restore_%s.ipynb" % (timestamp, uuid_str)
        try:
            notebook = pm.execute_notebook("RestoreBackup.ipynb",
                                output,
                                {
                                 "USER": user,
                                 "PASS": passw,
                                 "DIR": dir,
                                 "TAG": tag,
                                 "DEBUG": True}
                               )
        except pm.PapermillExecutionError:
            print("failed to run %s" %(dir_name))
            if DEBUG:
                print(uuid_str, '=>', output)
                print(str(notebook))
            sys.exit(1)


def cleanup():
    '''
        DESCRIPTION: cleanup pyc, .DS_Store, __pycache__ files and
               prune all images unused
        PARAM: None
    '''
    status = os.system(r'''
                find . -iname \*pyc -exec rm {} \;
                find . -iname .DS_Store -exec rm {} \;
                find . -iname __pycache__ -exec rmdir {} \;
                docker system prune -af;
                ''')
    assert status == 0, "failed"


def backup_incre(dir_list, user="", passw=""):
    '''
        DESCRIPTION: use docker-compose to do incremental and faster
                backup.  Must have INCR_EXPERIMENTAL as True to
                speedup. Uses latest tag to speed up
        PARAM: None
        RETURN: if success returns 0, otherwise non-zero
    '''
    try:
        backup(dir_list, is_full=False, generate=True, user=user, passw=passw)
    except pm.PapermillException as err:
        print(str(err))
        return -1
    return 0


def widget_run_default_backup(pos, blist):
    '''
        DESCRIPTION: use button to trigger backup from the blist selected
        PARAM: None
    '''

    selected = blist.trait_values()['value']
    if DEBUG:
        print(selected)
    pos.set_trait('button_style', 'info')
    pos.set_trait('description', 'In progress...')

    if backup_incre(selected) == 0:
        pos.set_trait('button_style', 'success')
        pos.set_trait('description', 'Done')
    else:
        pos.set_trait('button_style', 'danger')
        pos.set_trait('description', 'Failed')


In [58]:
    backup(["Tests"], user=USER, passw=PASS)


Tests is done


In [59]:
%gcrypt /Users/kenneyhe/Desktop/data-science lock --force
!cat secrets.yaml

è"�U�Y ��D�6���q��>�U�c���咂Pn���[=�j��G89�(3Ё¼�LiVV�<

In [146]:
%gcrypt /Users/kenneyhe/Desktop/data-science status

not encrypted: .ipynb_checkpoints/Api-checkpoint.ipynb
not encrypted: .ipynb_checkpoints/BackupHelper-checkpoint.ipynb
not encrypted: .gitattributes
not encrypted: .gitignore
not encrypted: .ipynb_checkpoints/Init-checkpoint.ipynb
not encrypted: BackupHelper.ipynb
not encrypted: Init.ipynb
not encrypted: LICENSE
not encrypted: README.md
not encrypted: Tests/Dockerfile
not encrypted: Tests/Widget%20List.ipynb
not encrypted: api.py
    encrypted: secrets.yaml


# Setup pre-commit

In [32]:
%%writefile .pre-commit-config.yaml
repos:
-   repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v3.4.0
    hooks:
    -   id: check-json
    -   id: check-yaml
    -   id: trailing-whitespace
    -   id: end-of-file-fixer
-   repo: https://github.com/pre-commit/mirrors-pylint
    rev: ''  # Use the sha / tag you want to point at
    hooks:
    -   id: pylint
-   repo: https://github.com/pre-commit/mirrors-eslint
    rev: v7.17.0
    hooks:
    -   id: eslint
        args: [--fix]

Overwriting .pre-commit-config.yaml


In [33]:
%%writefile .github/workflows/pre-commit.yml 
name: pre-commit

on:
  pull_request:
  push:
    branches:
    - main
    - master
jobs:
  pre-commit:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v2
    - uses: actions/setup-python@v2
    - uses: pre-commit/action@v2.0.0
      with:
        extra_args: eslint

Overwriting .github/workflows/pre-commit.yml


In [ ]:
%%bash
pre-commit install
pre-commit run --all-files
